# Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

#Read dataset 


---
This code snippet loads data for the  dataset from a specified 
location using the load_from_tsfile_to_dataframe function from the sktime package. The training data is stored in the train_x variable as a numpy array of arrays. Similarly, the test data is stored in the test_x variable. The corresponding labels for the training and test data are stored in train_y and test_y variables respectively. The labels are converted to floats using the astype method. This code can be used as a starting point for any analysis or modeling tasks involving the Cricket dataset.


---



In [2]:
def read_data_and_labels(train_filename, test_filename):
    train_x = []
    train_y = []
    test_x = []
    test_y = []

    # Read training data and labels
    with open(train_filename, 'r') as train_file:
        train_reader = csv.reader(train_file, delimiter=',')
        for row in train_reader:
            if row and not row[0].startswith(('#', '@')):
                label = row[-1].split(':')[-1]  # Extract label from the value after colon
                train_x.append([float(x.split(':')[0]) for x in row[:-1]])  # Remove colon and convert elements to float
                train_y.append(label)

    # Read testing data and labels
    with open(test_filename, 'r') as test_file:
        test_reader = csv.reader(test_file, delimiter=',')
        for row in test_reader:
            if row and not row[0].startswith(('#', '@')):
                label = row[-1].split(':')[-1]  # Extract label from the value after colon
                test_x.append([float(x.split(':')[0]) for x in row[:-1]])  # Remove colon and convert elements to float
                test_y.append(label)

    return train_x, train_y, test_x, test_y


In [ ]:
import csv
#@Usage:
train_filename = '/kaggle/input/multivariate2018-python/Multivariate_ts/InsectWingbeat/InsectWingbeat_TRAIN.ts'
test_filename = '/kaggle/input/multivariate2018-python/Multivariate_ts/InsectWingbeat/InsectWingbeat_TEST.ts'
train_x, train_y, test_x, test_y = read_data_and_labels(train_filename, test_filename)

In [ ]:
train_y = np.array(train_y)
train_x = np.array(train_x)
test_y = np.array(test_y)
test_x = np.array(test_x)

# apply pca 

The above code represents a data preprocessing pipeline for the cricket dataset.


---


The preprocess_data() function takes in train_x and test_x as inputs, which are the training and testing feature sets respectively. These feature sets are first standardized using the StandardScaler() function from scikit-learn's preprocessing module. Standardization involves centering the data and scaling it to unit variance. This is done to ensure that all features are on the same scale, which is important for many machine learning algorithms.


---


Next, Principal Component Analysis (PCA) is applied to the standardized data using the PCA() function from scikit-learn's decomposition module. PCA is a dimensionality reduction technique that transforms the original feature space into a new space of lower dimensionality. This is done by identifying the principal components, which are linear combinations of the original features that explain the most variance in the data.

---

The n_components parameter specifies the number of principal components to keep. In this case, it is set to 5. Finally, the preprocessed training and testing feature sets are returned.

---

Overall, this data preprocessing pipeline can help improve the performance of machine learning models on the cricket dataset by reducing the dimensionality of the feature space and scaling the data to a common range.

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def preprocess_data(train_x, test_x, n_components=5):
    # Convert train_x and test_x to 2D arrays with padding
    train_x_padded = np.array([x + [0] * (max_len - len(x)) for x in train_x])
    test_x_padded = np.array([x + [0] * (max_len - len(x)) for x in test_x])

    # Convert train_x and test_x to DataFrames
    train_x_df = pd.DataFrame(train_x_padded)
    test_x_df = pd.DataFrame(test_x_padded)

    # Scale the data
    scaler = StandardScaler()
    train_x_scaled = scaler.fit_transform(train_x_df)
    test_x_scaled = scaler.transform(test_x_df)

    # Apply PCA
    pca = PCA(n_components=n_components)
    train_x_pca = pca.fit_transform(train_x_scaled)
    test_x_pca = pca.transform(test_x_scaled)

    return train_x_pca, test_x_pca

# Determine the maximum length of sequences in train_x and test_x
max_len = max(max(len(x) for x in train_x), max(len(x) for x in test_x))

train_x, test_x = preprocess_data(train_x, test_x, n_components=5)

In [ ]:
print(train_y)
print(test_y)

from sklearn.preprocessing import LabelEncoder

# Create LabelEncoder object
label_encoder = LabelEncoder()

# Fit label encoder and transform train_y
train_y = label_encoder.fit_transform(train_y)
# Fit label encoder and transform train_y
test_y = label_encoder.fit_transform(test_y)


print(train_y)
print(test_y)


# Convert labels to one-hot encoding
from keras.utils import to_categorical
train_y = np.array(train_y)
test_y = np.array(test_y)
train_y = train_y.astype(int)
test_y = test_y.astype(int)

The code snippet you provided shows how to convert integer labels to one-hot encoded labels using the to_categorical function from Keras.


---


One-hot encoding is a technique used to represent categorical variables as binary vectors. In the context of classification tasks, this is often used to represent the target variable, where each category is assigned a unique binary vector. Each binary vector has a length equal to the total number of categories, and a value of 1 is assigned to the index corresponding to the category, while all other values are 0.

In [ ]:

# Convert labels to one-hot encoding
from keras.utils import to_categorical
train_y = np.array(train_y)
test_y = np.array(test_y)
train_y = train_y.astype(int)
test_y = test_y.astype(int)
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

In [ ]:
print(train_y.shape)
print(train_x.shape)

print(test_y.shape)
print(test_x.shape)




---
This line of code calculates the number of classes in the dataset. It uses 
the shape attribute of the numpy array train_y to get the number of rows (which represents the number of instances in the dataset) and the number of columns (which represents the number of classes). Since the dataset uses one-hot encoding for the labels, each row of train_y contains only one non-zero value, which corresponds to the class label of that instance. Therefore, the number of columns in train_y is equal to the number of classes in the dataset. The variable num_classes is assigned this value, which will be used later in the code for defining the output layer of the neural network model.


---




In [ ]:
num_classes = (train_y.shape[1])

#Resnet 

This is a function that defines a ResNet model using Keras. It takes two arguments: input_shape, which specifies the shape of the input data, and num_classes, which specifies the number of classes to predict. The model has four convolutional blocks, each consisting of a convolutional layer, batch normalization, ReLU activation, and dropout. The first three blocks have 64, 128, and 256 filters, respectively, while the fourth block has 512 filters. A shortcut connection is added from the input layer to the output of the fourth block, and the output is then passed through a final ReLU activation and flattened before being passed through a fully connected softmax layer with num_classes outputs. The model is compiled with the Adam optimizer, categorical crossentropy loss, and accuracy metric. This ResNet implementation is a modified version of the original ResNet architecture proposed by He et al. (2015) and is commonly used for time series classification tasks.

In [ ]:
from keras.initializers import *

def resnet(input_shape, num_classes, initializer):
    # Define the input layer
    input_layer = Input(shape=input_shape)

    # First convolutional block
    conv1 = Conv1D(filters=64, kernel_size=3, padding='same', kernel_initializer=initializer)(input_layer)
    norm1 = BatchNormalization()(conv1)
    relu1 = Activation('relu')(norm1)
    drop1 = Dropout(0.2)(relu1)

    # Second convolutional block
    conv2 = Conv1D(filters=128, kernel_size=3, padding='same', kernel_initializer=initializer)(drop1)
    norm2 = BatchNormalization()(conv2)
    relu2 = Activation('relu')(norm2)
    drop2 = Dropout(0.2)(relu2)

    # Third convolutional block
    conv3 = Conv1D(filters=256, kernel_size=3, padding='same', kernel_initializer=initializer)(drop2)
    norm3 = BatchNormalization()(conv3)
    relu3 = Activation('relu')(norm3)
    drop3 = Dropout(0.2)(relu3)

    # Fourth convolutional block
    conv4 = Conv1D(filters=512, kernel_size=3, padding='same', kernel_initializer=initializer)(drop3)
    norm4 = BatchNormalization()(conv4)
    relu4 = Activation('relu')(norm4)
    drop4 = Dropout(0.2)(relu4)

    # Add shortcut connection
    shortcut = Conv1D(filters=512, kernel_size=1, padding='same', kernel_initializer=initializer)(input_layer)
    shortcut = BatchNormalization()(shortcut)
    output = Add()([drop4, shortcut])
    output = Activation('relu')(output)

    # Flatten and output
    flatten = Flatten()(output)
    dense = Dense(num_classes, activation='softmax')(flatten)

    # Create and compile the model
    model = Model(inputs=input_layer, outputs=dense)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, Flatten, Dense, Dropout
from keras.regularizers import l2
initializers = [GlorotUniform(), HeUniform(), LecunUniform(), RandomUniform(), Zeros()]

# Create subplots for accuracy and loss
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))

for initializer in initializers:
    # Reshape the input data
    train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
    test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))

    # Define and train the model with the current initializer
    model = resnet(input_shape=(train_x.shape[1],1), num_classes=num_classes, initializer=initializer)
    history = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=100, batch_size=32)
    loss, accuracy = model.evaluate(test_x, test_y)
    print(f"Initializer: {str(initializer)}")
    print(f"Test loss: {loss}")
    print(f"Test accuracy: {accuracy}")
    print()

    # Plot accuracy graph
    ax1.plot(history.history['accuracy'], label=str(initializer))
    ax1.plot(history.history['val_accuracy'], label=str(initializer) + ' (val)')
    ax1.set_title('Model accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend(loc='lower right')

    # Plot loss graph
    ax2.plot(history.history['loss'], label=str(initializer))
    ax2.plot(history.history['val_loss'], label=str(initializer) + ' (val)')
    ax2.set_title('Model loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend(loc='upper right')

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()


#Tapent

The tapnet function defines a neural network model using convolutional layers for feature extraction and a dense layer for classification. The model has three convolutional blocks, each consisting of a 1D convolutional layer with ReLU activation followed by a max pooling layer. The output of the last convolutional block is flattened and fed into a dense layer with softmax activation, which produces the output probabilities for each class. The model is compiled with Adam optimizer and categorical cross-entropy loss function. This architecture is often used for audio processing tasks such as speech recognition and environmental sound classification.

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, concatenate, Dropout
from keras.initializers import GlorotUniform, HeUniform, LecunUniform, RandomUniform, Zeros
import matplotlib.pyplot as plt

def tapnet(input_shape, num_classes, initializer):
    # Define the input layer
    input_layer = Input(shape=input_shape)

    # First convolutional block
    conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_initializer=initializer)(input_layer)
    pool1 = MaxPooling1D(pool_size=2)(conv1)

    # Second convolutional block
    conv2 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same', kernel_initializer=initializer)(pool1)
    pool2 = MaxPooling1D(pool_size=2)(conv2)

    # Third convolutional block
    conv3 = Conv1D(filters=256, kernel_size=3, activation='relu', padding='same', kernel_initializer=initializer)(pool2)

    # Flatten the output from the convolutional layers
    flatten = Flatten()(conv3)

    # Define the output layer
    output_layer = Dense(units=num_classes, activation='softmax')(flatten)

    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
initializers = [GlorotUniform(), HeUniform(), LecunUniform(), RandomUniform(), Zeros()]

# Create subplots for accuracy and loss
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))

for initializer in initializers:
    # Reshape the input data
    train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
    test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))

    # Define and train the model with the current initializer
    model = tapnet(input_shape=(train_x.shape[1],1), num_classes=num_classes, initializer=initializer)
    history = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=100, batch_size=6)
    loss, accuracy = model.evaluate(test_x, test_y)
    print(f"Initializer: {str(initializer)}")
    print(f"Test loss: {loss}")
    print(f"Test accuracy: {accuracy*100}")
    print()

    # Plot accuracy graph
    ax1.plot(history.history['accuracy'], label=str(initializer))
    ax1.plot(history.history['val_accuracy'], label=str(initializer) + ' (val)')
    ax1.set_title('Model accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend(loc='lower right')

    # Plot loss graph
    ax2.plot(history.history['loss'], label=str(initializer))
    ax2.plot(history.history['val_loss'], label=str(initializer) + ' (val)')
    ax2.set_title('Model loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend(loc='upper right')

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

# CIF

the model architecture in the code you provided is a variant of the CIF (Convolutional-Inception-FullyConnected) model, which combines convolutional and fully connected layers to achieve high performance in image classification tasks.


---

This is a Convolutional Neural Network (CNN) model that has three convolutional blocks, followed by a GlobalAveragePooling1D layer and a dense output layer with a sigmoid activation function. It has been trained using the Adam optimizer and categorical cross-entropy loss function, and evaluated on a test set using accuracy as the evaluation metric.

The model architecture is suitable for a 1D Convolutional model to classify time-series data. The input shape is defined as (5, 1), meaning that the model takes in sequences of length 5 with a single feature dimension. The first convolutional block has 32 filters with a kernel size of 3, followed by another identical convolutional layer and a 25% dropout rate. The second convolutional block has 64 filters with a kernel size of 3, followed by another identical convolutional layer and a 25% dropout rate. The third convolutional block has 128 filters with a kernel size of 3, followed by another identical convolutional layer and a 25% dropout rate. The output of the third convolutional block is then passed through a GlobalAveragePooling1D layer to produce a fixed-size output. Finally, a dense output layer with a sigmoid activation function is used to produce the classification output.


In [ ]:
from keras.layers import Input, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense

def CIF(input_shape, num_classes, initializer):
    # Define the input layer
    input_layer = Input(shape=input_shape)

    # First convolutional block
    conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_initializer=initializer)(input_layer)
    pool1 = MaxPooling1D(pool_size=2)(conv1)

    # Second convolutional block
    conv2 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same', kernel_initializer=initializer)(pool1)
    pool2 = MaxPooling1D(pool_size=2)(conv2)

    # Third convolutional block
    conv3 = Conv1D(filters=256, kernel_size=3, activation='relu', padding='same', kernel_initializer=initializer)(pool2)

    # Flatten the output from the convolutional layers
    flatten = Flatten()(conv3)

    # Define the output layer
    output_layer = Dense(units=num_classes, activation='softmax')(flatten)

    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model



In [ ]:

initializers = [GlorotUniform(), HeUniform(), LecunUniform(), RandomUniform(), Zeros()]

# Create subplots for accuracy and loss
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))

for initializer in initializers:
    # Reshape the input data
    train_x_reshaped = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
    test_x_reshaped = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))

    # Define and train the model with the current initializer
    model = CIF(input_shape=(train_x_reshaped.shape[1], train_x_reshaped.shape[2]), num_classes=num_classes, initializer=initializer)
    history = model.fit(train_x_reshaped, train_y, validation_data=(test_x_reshaped, test_y), epochs=100, batch_size=6)
    loss, accuracy = model.evaluate(test_x_reshaped, test_y)
    print(f"Initializer: {str(initializer)}")
    print(f"Test loss: {loss}")
    print(f"Test accuracy: {accuracy*100}")
    print()

    # Plot accuracy graph
    ax1.plot(history.history['accuracy'], label=str(initializer))
    ax1.plot(history.history['val_accuracy'], label=str(initializer) + ' (val)')
    ax1.set_title('Model accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend(loc='lower right')

    # Plot loss graph
    ax2.plot(history.history['loss'], label=str(initializer))
    ax2.plot(history.history['val_loss'], label=str(initializer) + ' (val)')
    ax2.set_title('Model loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend(loc='upper right')

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

# INCEPTION

This code defines an Inception model for 1D convolutional neural networks in Keras. The model architecture is based on the Inception module, which uses multiple convolutional filters with different kernel sizes and max pooling to capture different levels of abstraction in the input data.


---


The inception_module function defines a single Inception module, which takes an input tensor x and applies four different convolutional filters with different kernel sizes (1, 3, and 5) and max pooling. The outputs of these filters are then concatenated along the feature dimension, resulting in a tensor with higher dimensionality.


---


The create_inception_model function uses the inception_module function to build a deep Inception model. The input shape and number of output classes are specified as arguments. The function builds a series of Inception modules with increasing filter sizes, followed by a global average pooling layer and a dense output layer with softmax activation. The model is compiled with categorical cross-entropy loss and the Adam optimizer.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.initializers import GlorotUniform, HeUniform, LecunUniform, RandomUniform, Zeros

def inception_module(x, filter_size, initializer):
    conv1 = layers.Conv1D(filter_size, kernel_size=1, padding='same', activation='relu', kernel_initializer=initializer)(x)
    conv3 = layers.Conv1D(filter_size, kernel_size=3, padding='same', activation='relu', kernel_initializer=initializer)(x)
    conv5 = layers.Conv1D(filter_size, kernel_size=5, padding='same', activation='relu', kernel_initializer=initializer)(x)
    max_pool = layers.MaxPooling1D(pool_size=3, strides=1, padding='same')(x)
    concat = layers.concatenate([conv1, conv3, conv5, max_pool], axis=-1)
    return concat

def create_inception_model(input_shape, num_classes, initializer):
    inputs = layers.Input(shape=input_shape)
    x = inputs
    x = inception_module(x, 32, initializer)
    x = inception_module(x, 64, initializer)
    x = inception_module(x, 128, initializer)
    x = inception_module(x, 256, initializer)
    x = layers.Flatten()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:

# Define the initializers
initializers = [GlorotUniform(), HeUniform(), LecunUniform(), RandomUniform(), Zeros()]

# Reshape the input data
train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))

# Create subplots for accuracy and loss
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))

for initializer in initializers:
    # Create the Inception-like model with the current initializer
    model = create_inception_model(input_shape=train_x.shape[1:], num_classes=num_classes, initializer=initializer)

    # Train the model for 100 epochs
    history = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=100, batch_size=32)

    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(test_x, test_y)
    print(f"Initializer: {str(initializer)}")
    print(f"Test loss: {loss}")
    print(f"Test accuracy: {accuracy*100}")
    print()

    # Plot accuracy graph
    ax1.plot(history.history['accuracy'], label=str(initializer))
    ax1.plot(history.history['val_accuracy'], label=str(initializer) + ' (val)')
    ax1.set_title('Model accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend(loc='lower right')

    # Plot loss graph
    ax2.plot(history.history['loss'], label=str(initializer))
    ax2.plot(history.history['val_loss'], label=str(initializer) + ' (val)')
    ax2.set_title('Model loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend(loc='upper right')

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

# Rocket 

The Rocket model is a simple yet effective 1D Convolutional Neural Network (CNN) architecture designed for time series classification tasks. The model is based on the idea of Random Convolutional Kernels (RCKs), which are small filters randomly generated from the input data to capture local patterns.

The Rocket model consists of five convolutional blocks, each with a different number of filters and kernel size, and a final dense layer for classification. The convolutional blocks are designed to learn increasingly complex representations of the input data, while the dropout layers help prevent overfitting.

The model uses batch normalization to normalize the activations between layers and to speed up the training process. Additionally, the model applies L2 regularization to the convolutional layers to prevent overfitting.

Overall, the Rocket model is a simple, yet effective architecture for time series classification tasks, and it has been shown to achieve state-of-the-art performance on several benchmark datasets. Its simplicity and efficiency make it a good candidate for real-world applications where computational resources are limited.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, Flatten, Dense, Dropout
from keras.regularizers import l2
from keras.initializers import GlorotUniform, HeUniform, LecunUniform, RandomUniform, Zeros

def rocket(input_shape, num_classes, initializer):
    # Define the input layer
    input_layer = Input(shape=input_shape)

    # First convolutional block
    conv1 = Conv1D(filters=16, kernel_size=11, padding='same', kernel_regularizer=l2(1e-4), kernel_initializer=initializer)(input_layer)
    norm1 = BatchNormalization()(conv1)
    relu1 = Activation('relu')(norm1)
    drop1 = Dropout(0.2)(relu1)

    # Second convolutional block
    conv2 = Conv1D(filters=32, kernel_size=9, padding='same', kernel_regularizer=l2(1e-4), kernel_initializer=initializer)(drop1)
    norm2 = BatchNormalization()(conv2)
    relu2 = Activation('relu')(norm2)
    drop2 = Dropout(0.2)(relu2)

    # Third convolutional block
    conv3 = Conv1D(filters=64, kernel_size=7, padding='same', kernel_regularizer=l2(1e-4), kernel_initializer=initializer)(drop2)
    norm3 = BatchNormalization()(conv3)
    relu3 = Activation('relu')(norm3)
    drop3 = Dropout(0.2)(relu3)

    # Fourth convolutional block
    conv4 = Conv1D(filters=128, kernel_size=5, padding='same', kernel_regularizer=l2(1e-4), kernel_initializer=initializer)(drop3)
    norm4 = BatchNormalization()(conv4)
    relu4 = Activation('relu')(norm4)
    drop4 = Dropout(0.2)(relu4)

    # Fifth convolutional block
    conv5 = Conv1D(filters=256, kernel_size=3, padding='same', kernel_regularizer=l2(1e-4), kernel_initializer=initializer)(drop4)
    norm5 = BatchNormalization()(conv5)
    relu5 = Activation('relu')(norm5)
    drop5 = Dropout(0.2)(relu5)

    # Flatten and output
    flatten = Flatten()(drop5)
    dense = Dense(num_classes, activation='softmax')(flatten)

    # Create and compile the model
    model = Model(inputs=input_layer, outputs=dense)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [ ]:

# Define the initializers
initializers = [GlorotUniform(), HeUniform(), LecunUniform(), RandomUniform(), Zeros()]

# Reshape the input data
train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))

# Create subplots for accuracy and loss
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))

for initializer in initializers:
    # Define and train the model with the current initializer
    model = rocket(input_shape=(train_x.shape[1], 1), num_classes=num_classes, initializer=initializer)
    history = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=100, batch_size=32)
    loss, accuracy = model.evaluate(test_x, test_y)
    print(f"Initializer: {str(initializer)}")
    print(f"Test loss: {loss}")
    print(f"Test accuracy: {accuracy*100}")
    print()

    # Plot accuracy graph
    ax1.plot(history.history['accuracy'], label=str(initializer))
    ax1.plot(history.history['val_accuracy'], label=str(initializer) + ' (val)')
    ax1.set_title('Model accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend(loc='lower right')

    # Plot loss graph
    ax2.plot(history.history['loss'], label=str(initializer))
    ax2.plot(history.history['val_loss'], label=str(initializer) + ' (val)')
    ax2.set_title('Model loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend(loc='upper right')

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()
